# Product Description Generation

## Install Packages

In [2]:
import sys
!{sys.executable} -m pip install -r ../requirements.txt

  Using cached numpy-1.18.3-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Using cached pandas-1.0.3-cp38-cp38-manylinux1_x86_64.whl (10.0 MB)
     |████████████████████████████████| 753.2 MB 31 kB/s s eta 0:00:01     |████████████████                | 378.4 MB 12.3 MB/s eta 0:00:31     |████████████████▏               | 381.2 MB 12.3 MB/s eta 0:00:31     |█████████████████████████▏      | 593.0 MB 8.7 MB/s eta 0:00:19██████▏    | 638.2 MB 5.4 MB/s eta 0:00:22     |█████████████████████████████▉  | 702.1 MB 13.6 MB/s eta 0:00:04
ERROR: Could not find a version that satisfies the requirement torchvision==0.4.0 (from -r ../requirements.txt (line 4)) (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.5.0, 0.6.0, 0.6.1, 0.7.0)
ERROR: No matching distribution found for torchvision==0.4.0 (from -r ../requirements.txt (line 4))


## Initialization
### Load packages

In [2]:
import pandas as pd
import numpy as np
import os
import re
import string
import torch
import torch.nn as nn

# Load helper files
sys.path.append(os.path.abspath('../'))
import helper
import network

### Load data

In [3]:
df = pd.read_csv('asics-data.csv')

/home/leonard/Workspace/development/research/product-attribute-extractor/pae-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,11,12,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.columns.values.tolist()

['ID',
 'Name',
 'Size (EU)',
 'Size (EU).1',
 'Size (EU).2',
 'Size (UK)',
 'Size (US)',
 'Size (JP)',
 'Shoe Width',
 'Color Code',
 'Brand',
 'Gender',
 'Season',
 'SAP Material',
 'Product Division',
 'Style Name (Dutch)',
 'Style Name',
 'Style Name (French)',
 'Style Name (German)',
 'Style Name (Italian)',
 'Style Name (Spanish)',
 'Color Name',
 'Color Name (Dutch)',
 'Color Name (French)',
 'Color Name (German)',
 'Color Name (Italian)',
 'Color Name (Spanish)',
 'Long Description (Dutch)',
 'Long Description',
 'Long Description (French)',
 'Long Description (German)',
 'Long Description (Italian)',
 'Long Description (Spanish)',
 'Page Description (Dutch)',
 'Page Description',
 'Page Description (French)',
 'Page Description (German)',
 'Page Description (Italian)',
 'Page Description (Spanish)',
 'Short Description (Dutch)',
 'Short Description',
 'Short Description (French)',
 'Short Description (German)',
 'Short Description (Italian)',
 'Short Description (Spanish)',
 '

In [5]:
#df[['Short Description', 'Long Description', 'Page Description']].head

#text = '\n'.join(df[['Short Description', 'Long Description', 'Page Description']])
#text = '\n'.join(df['Long Description'].astype(str))
#text = '\n'.join(df[df['Long Description'].str.len() > 3])

text = '\n'.join(df.loc[df['Long Description'].str.len() > 3]['Long Description'].drop_duplicates())
#text += '\n'.join(df.loc[df['Short Description'].str.len() > 3]['Short Description'].drop_duplicates())
#text += '\n'.join(df.loc[df['Page Description'].str.len() > 3]['Page Description'].drop_duplicates())

### Explore data

In [6]:
print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

helper.view_lines(text, 0, 5)

Dataset Stats
Roughly the number of unique words: 23458
Number of lines: 10953
Average number of words in each line: 40.30722176572628

The lines 0 to 5:
<p>Layer this bra with your gym kit favourites for a striking look that’s also comfortable. Feel flawless with Motion Dry technology to keep sweat away from your skin.</p>

<p>The cutout back begs to be seen, so wear under low-backed or racer-back tops for a stylish touch. Feel fantastic in soft fabric that stretches to support you whether you’re hitting the HIITs or swinging kettlebells.</p>
<ul>
	<li>Feel comfortable in soft, flexible fabric</li>


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [7]:
# cleaning data

# Remove tags
#text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)
text = helper.clean_html(text)
text = helper.strip_whitespaces(text)

# Replace special characters
text = helper.strip_accents(text)

helper.view_lines(text, 0, 5)


The lines 0 to 5:
Layer this bra with your gym kit favourites for a striking look that’s also comfortable. Feel flawless with Motion Dry technology to keep sweat away from your skin.

The cutout back begs to be seen, so wear under low-backed or racer-back tops for a stylish touch. Feel fantastic in soft fabric that stretches to support you whether you’re hitting the HIITs or swinging kettlebells.

 Feel comfortable in soft, flexible fabric


In [8]:
# pre-process training data
helper.preprocess_and_save_data(text)

# Check Point
If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [ ]:
# test dataloader
#helper.test_batch_data(range(50))

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [10]:
# Sequence Length
sequence_length = 15  # of words in a sequence
batch_size = 50

# data loader - do not change
train_loader = helper.batch_data(int_text, sequence_length, batch_size)

# Training parameters
num_epochs = 9
learning_rate = 0.001

# Model parameters
vocab_size = len(vocab_to_int)
output_size = len(vocab_to_int) # output is one word of the complete dictionary
embedding_dim = 450 # This should be OK, size vocab is not that long
hidden_dim = 400
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

# Training on GPU?
train_on_gpu = torch.cuda.is_available()
device = "cuda" if train_on_gpu else "cpu"
print("Running on: {}".format(device))

Running on: cuda


### STOP HERE

In [ ]:
# create model and move to gpu if available
rnn = network.RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu, dropout=0.5)
if rnn.train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = network.train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, train_loader, \
                                train_on_gpu, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training (cuda: True) for 9 epoch(s)...
Epoch:  1
Epoch:    1/9     Loss: 6.46771284866333


---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('trained_rnn')

## Generate Product Description
With the network trained and saved, you'll use it to generate a new, "fake" product descriptions.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!


In [ ]:
# run the cell multiple times to get different results!
gen_length = 100 # modify the length to your preference
prime_word = 'run' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = network.generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, \
                                    token_dict, vocab_to_int[pad_word], sequence_length, \
                                    train_on_gpu, gen_length)
print(generated_script)

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [ ]:
# save script to a text file
f =  open("./generated/script_rood_joris_1.txt","w")
f.write(generated_script)
f.close()